### Training Vlads model by first stacking the depths from the Sota models

In [1]:
import sys, os
import torch, wandb
import torch.nn as nn
from torch.utils.data import DataLoader
sys.path.append(os.path.abspath(os.path.join(os.curdir, '..')))
from configs import extended_unet_config as config
from models.unet_convnextv2 import Unet
from datasets.combination_depth_dataset import CombDepthDataset
from utils.train_utils import train_model
import utils.train_utils as tu
from utils.train_utils import evaluate_model
import importlib

/home/lucijatonkovic/miniforge3/envs/machine_perception/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lucijatonkovic/miniforge3/envs/machine_perception/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/lucijatonkovic/miniforge3/envs/machine_perception/lib/python3.12/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
# Set a fixed random seed for reproducibility
torch.manual_seed(config.random_seed)

train_full_dataset = CombDepthDataset(
    data_dir=os.path.join(config.dataset_path, 'train/train'),
    depths_dir=os.path.join(config.depth_maps_path, 'train'),
    list_file=os.path.join(config.dataset_path, 'train_list.txt'),
    transform=config.padded_transform,
    target_transform=config.target_transform,
    has_gt=True,
    depth_model_names=config.depth_model_names,
    uncertainty_dir=None,
    use_uncertainty=None)
    
    # Create test dataset without ground truth
test_dataset = CombDepthDataset(
    data_dir=os.path.join(config.dataset_path, 'test/test'),
    depths_dir=os.path.join(config.depth_maps_path, 'test'),
    list_file=os.path.join(config.dataset_path, 'test_list.txt'),
    transform=config.padded_transform,
    has_gt=False,
    depth_model_names=config.depth_model_names,
    uncertainty_dir=None,
    use_uncertainty=None)  # Test set has no ground truth
    
# Split training dataset into train and validation
total_size = len(train_full_dataset)
train_size = int((1-config.val_part) * total_size)  
val_size = total_size - train_size    
    
train_dataset, val_dataset = torch.utils.data.random_split(
    train_full_dataset, [train_size, val_size]
)
#val_dataset.transform = config.transform_val # I dont think we need to use augmentations for validation


# Create data loaders with memory optimizations
train_loader = DataLoader(
    train_dataset, 
    batch_size=config.train_bs, 
    shuffle=True, 
    num_workers=config.num_workers, 
    pin_memory=True,
    drop_last=True,
    persistent_workers=True,
)
    
    
val_loader = DataLoader(
    val_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)
    
test_loader = DataLoader(
    test_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)

print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")

Train size: 20375, Validation size: 3596, Test size: 650


In [3]:
model = config.model()


optimizer = config.optimizer(model.parameters())
print(f"Using device: {config.device}")

Num channels: 7
Using device: cpu


In [4]:
# Wandb logging
exp_name = "extended_unet_sota_depths"

print("Starting training...")
with wandb.init(project="MonocularDepthEstimation",  #CIL
                save_code=True,
                notes=config.WANDB_NOTES):
    model = train_model(model, train_loader, val_loader,
                        config.loss, optimizer, 1, config.device,
                       exp_path=os.path.join(config.dataset_path, exp_name),
                       is_extended_model=True, use_uncertainty_map=False)

Starting training...


wandb: Currently logged in as: ltonkovic (ltonkovic-eth-z-rich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/1


Training:   0%|          | 0/2546 [00:00<?, ?it/s]

RGB shape: torch.Size([8, 3, 448, 576])
Depth stack shape: torch.Size([8, 4, 448, 576])
Num of inputs 2
Inputs shape: torch.Size([8, 7, 448, 576])
Inputs shape: torch.Size([8, 7, 448, 576])


...

RGB shape: torch.Size([8, 3, 448, 576])
Depth stack shape: torch.Size([8, 4, 448, 576])
Num of inputs 2
Inputs shape: torch.Size([8, 7, 448, 576])
Inputs shape: torch.Size([8, 7, 448, 576])


Training:   0%|          | 12/2546 [03:58<14:01:02, 19.91s/it]
Traceback (most recent call last):
  File "/tmp/ipykernel_91391/386376533.py", line 8, in <module>
    model = train_model(model, train_loader, val_loader,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/lucijatonkovic/Documents/Git/CIL_2025/utils/train_utils.py", line 42, in train_model
    loss.backward()
  File "/home/lucijatonkovic/miniforge3/envs/machine_perception/lib/python3.12/site-packages/torch/_tensor.py", line 648, in backward
    torch.autograd.backward(
  File "/home/lucijatonkovic/miniforge3/envs/machine_perception/lib/python3.12/site-packages/torch/autograd/__init__.py", line 353, in backward
    _engine_run_backward(
  File "/home/lucijatonkovic/miniforge3/envs/machine_perception/lib/python3.12/site-packages/torch/autograd/graph.py", line 824, in _engine_run_backward
    return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
           ^^

MailboxClosedError: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f32cfe84e00>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f32cfbdfd40, execution_count=4 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 7f32cfbdfd10, raw_cell="# Wandb logging
exp_name = "extended_unet_sota_dep.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lucijatonkovic/Documents/Git/CIL_2025/notebooks/train_extended_unet.ipynb#X14sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe